In [1]:
import pandas as pd
import networkx as nx
import numpy as np

In [2]:
simplify_dir = 'GTFS/stop_snapping/GIS Simplify/'
gtfs_dir = 'GTFS/TTC_2016-10-03/'

In [3]:
stop_times_reduced = pd.read_csv(gtfs_dir + 'stop_times_reduced.csv')
stop_times_reduced

,trip_id,stop_sequence_new,stop_sequence,hr,min,INT_ID
0,32828689,1,1,5,15,13466640.0
1,32828689,2,2,5,16,13466531.0
2,32828689,3,3,5,16,13466677.0
3,32828689,4,4,5,17,13466754.0
4,32828689,5,5,5,18,13466699.0
...,...,...,...,...,...,...
3959705,32970576,7,8,25,56,13461602.0
3959706,32970576,8,9,25,57,13461912.0
3959707,32970576,9,10,25,57,13462284.0
3959708,32970576,10,11,25,58,13462427.0


In [4]:
hr_list = [7,8,9,10,11,12,13,14]
departure_hour = [7]

In [5]:
stop_times_reduced = stop_times_reduced[stop_times_reduced['hr'].isin(hr_list)]

stop_times_reduced['hr'] = np.where(stop_times_reduced['hr']>23, stop_times_reduced['hr'] - 24, stop_times_reduced['hr'])


/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_6195/1662348926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_times_reduced['hr'] = np.where(stop_times_reduced['hr']>23, stop_times_reduced['hr'] - 24, stop_times_reduced['hr'])


In [6]:
stop_times_reduced_o = stop_times_reduced.copy()
stop_times_reduced_d = stop_times_reduced.copy()
stop_times_reduced_d['stop_sequence_new'] = stop_times_reduced_d['stop_sequence_new'] - 1

In [7]:
stop_times_od = stop_times_reduced_o.merge(stop_times_reduced_d, left_on = ['trip_id', 'stop_sequence_new'],
                           right_on = ['trip_id', 'stop_sequence_new'], suffixes = ['_o', '_d'])

In [8]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d
0,32828692,1,2,9,9,13467129.0,3,9,10,13466778.0
1,32828692,2,3,9,10,13466778.0,4,9,11,13466640.0
2,32828692,3,4,9,11,13466640.0,5,9,12,13466531.0
3,32828692,4,5,9,12,13466531.0,6,9,13,13466677.0
4,32828692,5,6,9,13,13466677.0,7,9,15,13466754.0
...,...,...,...,...,...,...,...,...,...,...
1550350,32970403,45,48,8,59,13449259.0,49,9,0,13448840.0
1550351,32970403,46,49,9,0,13448840.0,51,9,1,13448292.0
1550352,32970403,47,51,9,1,13448292.0,52,9,2,13447884.0
1550353,32970403,48,52,9,2,13447884.0,54,9,4,13447218.0


In [9]:
stop_times_od['min_o'] = stop_times_od['min_o'].apply(lambda x: '{0:0>2}'.format(x))
stop_times_od['min_d'] = stop_times_od['min_d'].apply(lambda x: '{0:0>2}'.format(x))

In [10]:
stop_times_od['hr_o'] = stop_times_od['hr_o'].astype(str)
stop_times_od['hr_d'] = stop_times_od['hr_d'].astype(str)
# stop_times_od['min_o'] = stop_times_od['min_o'].astype(str)

In [11]:
stop_times_od['o_time_str'] = stop_times_od['hr_o'] + ':' + stop_times_od['min_o']
stop_times_od['d_time_str'] = stop_times_od['hr_d'] + ':' + stop_times_od['min_d']

In [12]:
stop_times_od['o_time'] = pd.to_datetime(stop_times_od['o_time_str'], format = '%H:%M')
stop_times_od['d_time'] = pd.to_datetime(stop_times_od['d_time_str'], format = '%H:%M')


In [13]:
stop_times_od['cost'] = ((stop_times_od['d_time'] - stop_times_od['o_time']).dt.seconds)/60

In [14]:
stop_times_od['o_time'] = stop_times_od['o_time'].dt.time
stop_times_od['d_time'] = stop_times_od['d_time'].dt.time

In [15]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,o_time_str,d_time_str,o_time,d_time,cost
0,32828692,1,2,9,09,13467129.0,3,9,10,13466778.0,9:09,9:10,09:09:00,09:10:00,1.0
1,32828692,2,3,9,10,13466778.0,4,9,11,13466640.0,9:10,9:11,09:10:00,09:11:00,1.0
2,32828692,3,4,9,11,13466640.0,5,9,12,13466531.0,9:11,9:12,09:11:00,09:12:00,1.0
3,32828692,4,5,9,12,13466531.0,6,9,13,13466677.0,9:12,9:13,09:12:00,09:13:00,1.0
4,32828692,5,6,9,13,13466677.0,7,9,15,13466754.0,9:13,9:15,09:13:00,09:15:00,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550350,32970403,45,48,8,59,13449259.0,49,9,00,13448840.0,8:59,9:00,08:59:00,09:00:00,1.0
1550351,32970403,46,49,9,00,13448840.0,51,9,01,13448292.0,9:00,9:01,09:00:00,09:01:00,1.0
1550352,32970403,47,51,9,01,13448292.0,52,9,02,13447884.0,9:01,9:02,09:01:00,09:02:00,1.0
1550353,32970403,48,52,9,02,13447884.0,54,9,04,13447218.0,9:02,9:04,09:02:00,09:04:00,2.0


In [16]:
trips_weekday = pd.read_csv(gtfs_dir + 'trips_weekday.csv')

In [17]:
stop_times_od = stop_times_od.merge(trips_weekday[['trip_id', 'route_short_name', 'direction_id']], how = 'inner')

In [18]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,o_time_str,d_time_str,o_time,d_time,cost,route_short_name,direction_id
0,32828692,1,2,9,09,13467129.0,3,9,10,13466778.0,9:09,9:10,09:09:00,09:10:00,1.0,506,0
1,32828692,2,3,9,10,13466778.0,4,9,11,13466640.0,9:10,9:11,09:10:00,09:11:00,1.0,506,0
2,32828692,3,4,9,11,13466640.0,5,9,12,13466531.0,9:11,9:12,09:11:00,09:12:00,1.0,506,0
3,32828692,4,5,9,12,13466531.0,6,9,13,13466677.0,9:12,9:13,09:12:00,09:13:00,1.0,506,0
4,32828692,5,6,9,13,13466677.0,7,9,15,13466754.0,9:13,9:15,09:13:00,09:15:00,2.0,506,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506362,32970403,45,48,8,59,13449259.0,49,9,00,13448840.0,8:59,9:00,08:59:00,09:00:00,1.0,35,1
506363,32970403,46,49,9,00,13448840.0,51,9,01,13448292.0,9:00,9:01,09:00:00,09:01:00,1.0,35,1
506364,32970403,47,51,9,01,13448292.0,52,9,02,13447884.0,9:01,9:02,09:01:00,09:02:00,1.0,35,1
506365,32970403,48,52,9,02,13447884.0,54,9,04,13447218.0,9:02,9:04,09:02:00,09:04:00,2.0,35,1


Inner merge because some trips in the stop times files have `service_id` 2 and 3 (meaning they run on weekends).

Type 1 nodes are boarding, while type 2 nodes are alighting. Type 0 nodes are regular waiting nodes. This is to penalize transfers, and make transfering more realistic. Links connecting type 1 and type 2 nodes to type 0 nodes will be added.

In [19]:
stop_times_od['node_o'] = stop_times_od['INT_ID_o'].astype(
    int).astype(str) + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-'+ stop_times_od['hr_o'].astype(
    str) + '-' + stop_times_od['min_o'].astype(int).astype(str)
stop_times_od['node_d'] = stop_times_od['INT_ID_d'].astype(
    int).astype(str) + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-'+ stop_times_od['hr_d'].astype(
    str) + '-' + stop_times_od['min_d'].astype(int).astype(str)

In [20]:
stop_times_od['link'] = '0' + '-' + stop_times_od['route_short_name'].astype(int).astype(str) +'-' + stop_times_od['trip_id'].astype(str) + '-' + stop_times_od['direction_id'].astype(str)+ '_' + stop_times_od['node_o'] + '_' + stop_times_od['node_d']

In [21]:
stop_times_od['cost'] = stop_times_od['cost'].astype(int)

In [22]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,o_time_str,d_time_str,o_time,d_time,cost,route_short_name,direction_id,node_o,node_d,link
0,32828692,1,2,9,09,13467129.0,3,9,10,13466778.0,9:09,9:10,09:09:00,09:10:00,1,506,0,13467129-506-9-9,13466778-506-9-10,0-506-32828692-0_13467129-506-9-9_13466778-506...
1,32828692,2,3,9,10,13466778.0,4,9,11,13466640.0,9:10,9:11,09:10:00,09:11:00,1,506,0,13466778-506-9-10,13466640-506-9-11,0-506-32828692-0_13466778-506-9-10_13466640-50...
2,32828692,3,4,9,11,13466640.0,5,9,12,13466531.0,9:11,9:12,09:11:00,09:12:00,1,506,0,13466640-506-9-11,13466531-506-9-12,0-506-32828692-0_13466640-506-9-11_13466531-50...
3,32828692,4,5,9,12,13466531.0,6,9,13,13466677.0,9:12,9:13,09:12:00,09:13:00,1,506,0,13466531-506-9-12,13466677-506-9-13,0-506-32828692-0_13466531-506-9-12_13466677-50...
4,32828692,5,6,9,13,13466677.0,7,9,15,13466754.0,9:13,9:15,09:13:00,09:15:00,2,506,0,13466677-506-9-13,13466754-506-9-15,0-506-32828692-0_13466677-506-9-13_13466754-50...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506362,32970403,45,48,8,59,13449259.0,49,9,00,13448840.0,8:59,9:00,08:59:00,09:00:00,1,35,1,13449259-35-8-59,13448840-35-9-0,0-35-32970403-1_13449259-35-8-59_13448840-35-9-0
506363,32970403,46,49,9,00,13448840.0,51,9,01,13448292.0,9:00,9:01,09:00:00,09:01:00,1,35,1,13448840-35-9-0,13448292-35-9-1,0-35-32970403-1_13448840-35-9-0_13448292-35-9-1
506364,32970403,47,51,9,01,13448292.0,52,9,02,13447884.0,9:01,9:02,09:01:00,09:02:00,1,35,1,13448292-35-9-1,13447884-35-9-2,0-35-32970403-1_13448292-35-9-1_13447884-35-9-2
506365,32970403,48,52,9,02,13447884.0,54,9,04,13447218.0,9:02,9:04,09:02:00,09:04:00,2,35,1,13447884-35-9-2,13447218-35-9-4,0-35-32970403-1_13447884-35-9-2_13447218-35-9-4


In [23]:
walk_links = pd.read_csv('GIS/int_tts_walk_time.csv')
walk_links['duration'] = round(walk_links['duration']/60).astype(int)
walk_links['gta06'] = walk_links['gta06'] + 1000
walk_links

,gta06,INT_ID,duration
0,1001,13461602,25
1,1001,13461733,21
2,1001,13462844,20
3,1001,13461863,27
4,1001,13463408,9
...,...,...,...
37157,1081,13467412,24
37158,1081,13467854,6
37159,1081,13466312,22
37160,1081,13467197,29


In [24]:
G = nx.DiGraph()

## Adding node-times

In [25]:
nodes = list((stop_times_od['INT_ID_o'].append(stop_times_od['INT_ID_d'])).drop_duplicates())

In [26]:
node_list = []
for node in nodes:
    for hr_i in hr_list:
        if hr_i > 23:  
            hr = hr_i - 24
        else:
            hr = hr_i
        for minute in range(60):
            node_id = str(int(node)) + '-0-' + str(hr) + '-' + str(minute)
            node_list.append(node_id)

In [27]:
G.add_nodes_from(node_list)

## Adding Boarding Nodes

In [28]:
boarding_nodes = list(stop_times_od['node_o'].drop_duplicates())

In [29]:
G.add_nodes_from(boarding_nodes)

## Adding Alighting Nodes

In [30]:
alighting_nodes = list(stop_times_od['node_d'].drop_duplicates())

In [31]:
G.add_nodes_from(alighting_nodes)

## Adding travel links

In [32]:
link_list = []
for index, row in stop_times_od.iterrows():
    attributes = {'cost': row['cost']}
    link = (row['node_o'], row['node_d'], attributes)
    link_list.append(link)

In [33]:
G.add_edges_from(link_list)

## Adding waiting links

In [34]:
transfer_list = []
for node in nodes:
    next_hr = hr_list[0]
    next_minute = 0
    node = int(node)
    for hr_i in hr_list:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
            if current_minute == 59:
                next_minute = 0
                next_hr = current_hr + 1
                
            else:
                next_minute = current_minute + 1
            node_o = str(node) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(node) + '-0-' + str(next_hr) + '-' + str(next_minute)
            
            link_name = '1-0-0-9_' + node_o + '_' + node_d
            
            attributes = {'cost': 1}
            
            
            link = (node_o, node_d, attributes)
            transfer_list.append(link)
    

In [35]:
G.add_edges_from(transfer_list)

## Adding Boarding Links

As a start, we'll match Google Maps minimum transfer time of 4 minutes, with 2 minute cost for both boarding and alighting links, meaning transfers be a minimum of 4 minutes.

A direction id of 1 means inbound to the stop node.

In [36]:
boarding_link = []
for node in boarding_nodes:
    node_split = node.split('-')
    int_id = node_split[0]
    boarding_hr = int(node_split[2])
    boarding_minute = int(node_split[3])

    stop_minute = boarding_minute - 2
    if stop_minute < 0:
        stop_hr = boarding_hr -1
        stop_minute = 60 + stop_minute
    else:
        stop_hr = boarding_hr

    stop_node = int_id + '-0-' + str(stop_hr) + '-' + str(stop_minute)
    attributes = {'cost': 2}
    
    link = (stop_node, node, attributes)
    
    boarding_link.append(link)

In [37]:
G.add_edges_from(boarding_link)

## Adding Alighting Links

In [38]:
alighting_link = []
for node in alighting_nodes:
    node_split = node.split('-')
    int_id = node_split[0]
    alighting_hr = int(node_split[2])
    alighting_minute = int(node_split[3])

    stop_minute = alighting_minute + 2
    if stop_minute > 59:
        stop_hr = alighting_hr + 1
        stop_minute = stop_minute - 60
    else:
        stop_hr = alighting_hr

    stop_node = int_id + '-0-' + str(stop_hr) + '-' + str(stop_minute)
    attributes = {'cost': 2}
    
    link = (node, stop_node, attributes)
    
    alighting_link.append(link)

In [39]:
G.add_edges_from(alighting_link)

## Adding TTS Zone Origins

Will also follow node-time structure

In [40]:
tts_zones = list(walk_links['gta06'].drop_duplicates())

In [41]:
node_stationary_list = []
for zone in tts_zones:
    for hr_i in hr_list:
        if hr_i > 23:  
            hr = hr_i - 24
        else:
            hr = hr_i
        for minute in range(60):
            node_id = str(int(zone)) + '-0-' + str(hr) + '-' + str(minute)
            node_stationary_list.append(node_id)

In [42]:
G.add_nodes_from(node_stationary_list)

## Adding TTS Zone Destinations

Only a single node for each zone

In [43]:
zone_destinations = []
for zone in tts_zones:
    node_id = str(int(zone)) + '-0-99-99'
    zone_destinations.append(node_id)

In [44]:
G.add_nodes_from(zone_destinations)

## Adding links from TTS to intersections

Both outbound and inbound links will be added. In pathfinding process, only inbound links for the destination, and outbound links for the origin will be kept.

Only links during the departure hour will be made due to space concerns.

Convention is 0 for outbound and 1 for inbound

In [45]:
walk_links.head()

,gta06,INT_ID,duration
0,1001,13461602,25
1,1001,13461733,21
2,1001,13462844,20
3,1001,13461863,27
4,1001,13463408,9


In [46]:
walk_link_outbound = []
for index, row in walk_links.iterrows():
    
    zone = int(row['gta06'])
    node = int(row['INT_ID'])
    cost = int(row['duration'])
    
    for hr_i in departure_hour:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
            next_hr = current_hr
            next_minute = current_minute + cost
            if next_minute > 59:
                next_hr= current_hr + 1
                next_minute = next_minute - 60
                
            node_o = str(zone) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(node) + '-0-' + str(next_hr) + '-' + str(next_minute)
            
            link_name = '2-0-0-0_' + node_o + '_' + node_d
            
            attributes = {'cost': cost}
    
            link = (node_o, node_d, attributes)
            walk_link_outbound.append(link)

## Adding links from intersections to TTS Zones

In [47]:
walk_link_inbound = []
for index, row in walk_links.iterrows():
    
    zone = int(row['gta06'])
    node = int(row['INT_ID'])
    cost = int(row['duration'])
    
    for hr_i in hr_list:
        if hr_i > 23:
            current_hr = hr_i - 24 
        else:
            current_hr = hr_i
        for minute in range(60):
            current_minute = minute
                
            node_o = str(node) + '-0-' + str(current_hr) + '-' + str(current_minute)
            node_d = str(zone) + '-0-99-99'
            
            link_name = '2-0-0-1_' + node_o + '_' + node_d
            
            attributes = {'cost': cost}
    
            link = (node_o, node_d, attributes)
            walk_link_inbound.append(link)

In [48]:
G.add_edges_from(walk_link_outbound)

In [49]:
G.add_edges_from(walk_link_inbound)

In [54]:
list(G.neighbors('151-0-7-4'))

['151-0-7-5',
 '151-1-7-6',
 '151-504-7-6',
 '151-514-7-6',
 '1015-0-99-99',
 '1016-0-99-99',
 '1024-0-99-99',
 '1025-0-99-99',
 '1026-0-99-99',
 '1028-0-99-99',
 '1030-0-99-99',
 '1031-0-99-99',
 '1032-0-99-99',
 '1033-0-99-99',
 '1034-0-99-99',
 '1035-0-99-99',
 '1036-0-99-99',
 '1037-0-99-99',
 '1038-0-99-99',
 '1040-0-99-99',
 '1048-0-99-99',
 '1049-0-99-99',
 '1050-0-99-99',
 '1051-0-99-99',
 '1052-0-99-99',
 '1053-0-99-99',
 '1054-0-99-99',
 '1055-0-99-99',
 '1056-0-99-99',
 '1057-0-99-99',
 '1058-0-99-99',
 '1059-0-99-99',
 '1060-0-99-99',
 '1061-0-99-99',
 '1062-0-99-99',
 '1063-0-99-99',
 '1064-0-99-99',
 '1065-0-99-99',
 '1066-0-99-99',
 '1067-0-99-99',
 '1068-0-99-99',
 '1069-0-99-99',
 '1076-0-99-99',
 '1077-0-99-99',
 '1078-0-99-99',
 '1079-0-99-99',
 '1080-0-99-99',
 '1090-0-99-99',
 '1081-0-99-99']

In [51]:
nx.write_graphml(G,'networks/AM-TE-16-8.graphml')
stop_times_od.to_csv('networks/AM-TE-16-8.csv', index = False)

In [52]:
G.number_of_nodes()

2488282

In [53]:
G.number_of_edges()

23203653